# Optimisation solutions
The optimum solution for each week differs. To improve converge quality and speed, the optimum for each week shall be calculated and stored

In [ ]:
# imports
import json
import os

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pqdm.processes import pqdm

from src.opt_utils import worker
from src.utils import WEEK_MAP

In [ ]:
# calculate output as dictionary
weeks = WEEK_MAP["week"].values
opt_init_weekly = {
    int(k): v
    for k, v in zip(
        weeks, pqdm(weeks, worker, n_jobs=int(3 / 4 * os.cpu_count()), leave=False)
    )
}

In [ ]:
# save file
with open(os.path.join(os.getcwd(), "weekly_optimum.json"), "w") as f:
    f.write(json.dumps(opt_init_weekly, indent=4))

In [ ]:
# plot results
df = pd.DataFrame(opt_init_weekly).T
fig, ax_all = plt.subplots(ncols=2, figsize=(12, 12))
for data, ax in zip([df.drop(columns="score"), df[["score"]]], ax_all):
    sns.boxplot(data=data, orient="h", ax=ax)
    ax.grid()
ax_all[0].set_xlim(left=0)
ax_all[1].set_xlim(left=0)
fig.tight_layout()